In [ ]:
import utility_module_2_v_3 as uu
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy.ma as ma
from numpy import meshgrid
from mpl_toolkits.basemap import Basemap
from tqdm import tqdm

import geopandas as gpd
from shapely.geometry  import Point
from shapely.geometry import shape
import seaborn as sns


In [ ]:
path_out_P = '/home/vikram/BCSD_data/' + 'US_P_BCSD/'
path_out_T = '/home/vikram/BCSD_data/' + 'US_T_BCSD/'

output_figname = ''#_eqm'

# init

In [ ]:
gcm_names=['MIROC_ESM','CanESM2','HadGEM_AO','GFDL_CM3']
gcm_name  =  gcm_names[1]

In [ ]:
def getclosest_ij(lats,lons,latpt,lonpt):
    dis_sq1=(lats-latpt)**2
    dis_sq2=(lons-lonpt)**2
    
    minidex_lat=dis_sq1.argmin()
    minidex_lon=dis_sq2.argmin()
    
    return minidex_lon,minidex_lat

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import geopandas as gpd
from geopy.geocoders import Nominatim



lats,lons = (np.arange(0,40,1),np.arange(-180,100,1))
XX12,YY12 = np.meshgrid(lons,lats)

fig, ax = plt.subplots()

for address in ['San Francisco','New York','Los Angeles', 'Chicago']:
    geolocator = Nominatim(user_agent="Your_Name")
    location = geolocator.geocode(address)
    [ind_lon,ind_lat] = getclosest_ij(lats,lons,location.latitude,location.longitude)
    plt.plot(lons[ind_lon],lats[ind_lat],marker='*',markersize= 25)

ind = gpd.read_file('/home/vikram/BCSD_Shapefiles/Us_boundary.shp')
ind.plot(ax=ax,facecolor="none")

regional_boxes = [[-120,37,-113,45],
                  [-109,37,-98,45],
                  [-88,33,-77,41]]

for co_ords in regional_boxes:
    ax.add_patch(Polygon([(co_ords[0],co_ords[1]),
                           (co_ords[0],co_ords[3]),
                           (co_ords[2],co_ords[3]),
                           (co_ords[2],co_ords[1])],facecolor='none',edgecolor= 'k'))

plt.show()




# Import data

In [ ]:

metadata=np.load(path_out_P+ '/pre_metadata'+gcm_name+'.npy', allow_pickle=True)
st=metadata.item()['st']
en=metadata.item()['en']

mask_fine_obs=metadata.item()['mask']

lat_obs=metadata.item()['lat_obs']
lon_obs=metadata.item()['lon_obs']

N_random = 10
gcm_list = ['GFDL_CM3','CanESM2','MIROC_ESM','MRI_ESM1','ACCESS1_0','NorESM1_M','CM5A_LR','CNRM_CM5','MPI_ESM_MR']
N_esms = len(gcm_list)

tas_obs=np.load(path_out_T+ '/tas_obs.npy')
tas_data_all=np.empty([N_esms + N_esms,95*12,tas_obs.shape[1],tas_obs.shape[2]])
pre_obs=np.load(path_out_P+ '/pre_obs.npy')
pre_data_all=np.empty([ N_esms+N_esms ,95*12,pre_obs.shape[1],pre_obs.shape[2]])


for giter,gcm_name in enumerate(gcm_list):
    
    tas_gcm=np.load(path_out_T+ '/tas_gcm_future_'+gcm_name+'.npy')
    metadata=np.load(path_out_P+ '/pre_metadata'+gcm_name+'.npy', allow_pickle=True)
    lat_gcm=metadata.item()['lat_gcm']
    lon_gcm=metadata.item()['lon_gcm']
    tas_gcm_fine=np.zeros((tas_gcm.shape[0],tas_obs.shape[1],tas_obs.shape[2]))
    for i in range(tas_gcm.shape[0]):
        tas_gcm_fine[i,:,:]=uu.regrid(tas_gcm[i,:,:],lat_gcm,lon_gcm,lat_obs,lon_obs)
    tas_data_all[giter ]        = tas_gcm_fine  
    temp_123 =(np.load(path_out_T+ '/detrended_BCSD_outputs_tas_'+gcm_name+'_cru.npy'))
    
    tas_data_all[giter + N_esms] = temp_123
      

    pre_gcm=np.load(path_out_P+ '/pre_gcm_future_'+gcm_name+'.npy')
    metadata=np.load(path_out_P+ '/pre_metadata'+gcm_name+'.npy', allow_pickle=True)
    lat_gcm=metadata.item()['lat_gcm']
    lon_gcm=metadata.item()['lon_gcm']
    pre_gcm_fine=np.zeros((pre_gcm.shape[0],pre_obs.shape[1],pre_obs.shape[2]))
    for i in range(pre_gcm.shape[0]):
        pre_gcm_fine[i,:,:]=uu.regrid(pre_gcm[i,:,:],lat_gcm,lon_gcm,lat_obs,lon_obs)
        
    pre_data_all[giter  ]=       pre_gcm_fine 
    pre_data_all[giter  + N_esms]= uu.correct_nan_inf_neg(np.load(path_out_P+ '/detrended_BCSD_outputs_pre_'+gcm_name+'_cru.npy')) 
                                    
        
        
        
        



# SPI

In [ ]:
import scipy.stats

def calc_spi_nonparam(data):
  N=data.shape[0]
  index_ts=np.arange(N)  

  data_non_zeros=(data[data>0])
  index_non_zeros=index_ts[data>0]
  N_non_zeros=len(data_non_zeros)
  index_sorted=  np.argsort(data_non_zeros)
  ranks = np.empty_like(index_sorted)
  ranks[index_sorted] = np.arange(N_non_zeros)

  p_gringorten=(ranks-0.44)/(N_non_zeros+0.12)
  p_g_all=np.zeros_like(data)
  p_g_all[data>0]=p_gringorten


  count_zeros=(data[data==0].shape[0])
  count_total=(data.shape[0])
  p_zeros=count_zeros/count_total

  cdf2 = p_zeros + (1 - p_zeros) * p_g_all
  SPI= scipy.stats.norm.ppf(cdf2)
  return SPI



In [ ]:
def extract_regions(data_input,lat_obs,lon_obs):
    
    data_to_indices = []
    for address in ['San Francisco','New York','Los Angeles', 'Chicago']:
        geolocator = Nominatim(user_agent="Your_Name")
        location = geolocator.geocode(address)
        [ind_lon,ind_lat] = getclosest_ij(lat_obs.data,lon_obs.data,location.latitude%360,location.longitude%360)
        data_to_indices.append(data_input[:, ind_lat,ind_lon])


    regional_boxes = [[-120%360,45,-113%360,37],
                      [-109%360,45,-98%360,37],
                      [-88%360,41,-77%360,33]]
    
    
    for co_ords in regional_boxes:

        lat_index_123 = np.logical_and(lat_obs >= co_ords[3], lat_obs <= co_ords[1])
        lon_index_123 = np.logical_and(lon_obs >= co_ords[0], lon_obs <= co_ords[2])
        # print(lat_index_123.sum(), lon_index_123.sum())
        data_to_indices.append(np.nanmean(data_input[:, lat_index_123,:][:,:,lon_index_123],axis=(1,2)))

    data_to_indices.append(np.nanmean(data_input,axis=(1,2)))
    
    return data_to_indices


In [ ]:
st = 2006

In [ ]:
import copy
N_datasets = N_esms *2 
N_regions = 8

dalta_tas_all = {}
delta_spi_all = {}
delta_tas_spi_all = {}

for diter in tqdm(range(N_datasets)):

    data_precipitation_all = extract_regions(pre_data_all[diter],lat_obs,lon_obs)
    data_temperature_all = extract_regions(tas_data_all[diter],lat_obs,lon_obs)
    
    dalta_tas_all[diter] = []
    delta_spi_all[diter] = []
    delta_tas_spi_all[diter] = []
    


    
    for riter in range(N_regions):
        precipitation_data = data_precipitation_all[riter]
        temperature_data  = data_temperature_all[riter]
        
        pre_6_monthly=copy.deepcopy(precipitation_data)
        pre_6_monthly[3:-2]  =   (precipitation_data[0:-5]+
                                  precipitation_data[1:-4]+
                                  precipitation_data[2:-3]+
                                  precipitation_data[3:-2]+
                                  precipitation_data[4:-1]+
                                  precipitation_data[5:])/6

        SPI_monthly_np=calc_spi_nonparam(pre_6_monthly.flatten())

        period_0=[2006,2036] 
        period_1=[2070,2100] 
        N_y = 30

        tp0=temperature_data[(period_0[0]-st)*12:(period_0[1]-st+1)*12]
        tp1=temperature_data[(period_1[0]-st)*12:(period_1[1]-st+1)*12]
        delta_tas = np.mean(tp1,axis=0)  -  np.mean(tp0,axis=0)

        spi0=SPI_monthly_np[(period_0[0]-st)*12:(period_0[1]-st+1)*12]
        spi1=SPI_monthly_np[(period_1[0]-st)*12:(period_1[1]-st+1)*12]
        d__02=np.all(np.array([spi0<=-0.8]),axis=0).sum()
        d__12=np.all(np.array([spi1<=-0.8]),axis=0).sum()
        delta_spi_freq = (d__12-d__02)/N_y

        tas_d1_p0=np.nanmean(tp0[np.all(np.array([spi0<=-0.8]),axis=0)])
        tas_d1_p1=np.nanmean(tp1[np.all(np.array([spi1<=-0.8]),axis=0)])
        delta_tas_d1  =   tas_d1_p1    -  tas_d1_p0

        
        dalta_tas_all[diter].append(delta_tas)
        delta_spi_all[diter].append(delta_spi_freq)
        delta_tas_spi_all[diter].append(delta_tas_d1)


In [ ]:

plt.figure(figsize = (8,4))

N_esm =len(gcm_list)
N_random = 10
    
markers=['o','^','s','p','P','*','h','+','x','X','D']
data_to_indices_label = ['San Francisco','New York','Los Angeles', 'Chicago', 'West', 'Middle', 'East' ,"CONUS"]


for pliter in range(8):
    plt.subplot(2,4,pliter +1)

    for gi in range(N_esm):
        plt.plot( 2 , dalta_tas_all[gi][pliter],
                 markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])

        plt.plot( 3 , dalta_tas_all[gi+N_esm][pliter],
                 markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])
        
        plt.plot( 4 , dalta_tas_all[gi+N_esm][pliter]- dalta_tas_all[gi][pliter],
                 markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])

    # plt.ylim([-1.5,1.5])
    

    plt.xticks(range(2,5),["ESM","BCSD","$\Delta$"],rotation = 90 )
    plt.title(data_to_indices_label[pliter])
    
    plt.xlim([0.5,4.5])
    plt.ylabel("$^{\circ}$C")
    
plt.tight_layout()
# plt.suptitle()|
# plt.savefig("./temp_plots/detrded_delta_tas.png", dpi =150, facecolor = 'w')


In [ ]:

plt.figure(figsize = (8,4))

N_esm =len(gcm_list)
N_random = 10
    
markers=['o','^','s','p','P','*','h','+','x','X','D']


for pliter in range(8):
    plt.subplot(2,4,pliter +1)

    for gi in range(N_esm):
        plt.plot( 2 , delta_tas_spi_all[gi][pliter],
                 markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])

        plt.plot( 3 , delta_tas_spi_all[gi+N_esm][pliter],
                 markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])
        
        plt.plot( 4 , delta_tas_spi_all[gi+N_esm][pliter]- delta_tas_spi_all[gi][pliter],
                 markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])

    # plt.ylim([-1.5,1.5])
    

    plt.xticks(range(2,5),["ESM","BCSD","$\Delta$"],rotation = 90 )
    plt.title(data_to_indices_label[pliter])
    
    plt.xlim([0.5,4.5])
    plt.ylabel("$^{\circ}$C")
    
plt.tight_layout()
# plt.savefig("./temp_plots/detded_delta_tas_spi.png", dpi =150, facecolor = 'w')


In [ ]:
N_esm =len(gcm_list)

markers=['o','^','s','p','P','*','h','+','x','X','D']


# data_to_indices_label  =  ['Kolkata', 'Mumbai', 'New Delhi' , 'Chennai' , 
#                            'Western India', 'Central India' , 'NE India','All India']

# plt.plot( 0,0, 'D',markersize = 10,color ='w',linestyle = 'None',mfc='none',label ='Random Fields')
plt.figure(figsize = (15,4))

for pliter in range(8):
    plt.subplot(2,4,pliter +1)
    plt.plot(1,dalta_tas_all[0][pliter], marker = markers[0],markersize = 10,linestyle = 'None',mfc='none',label ='Random Fields')

    for gi in range(N_esm):
        plt.plot( 2 , dalta_tas_all[gi+1][pliter],
                 markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])

        plt.plot( 3 , dalta_tas_all[gi+N_esm+1][pliter],
                 markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])
        
        plt.plot( 4 , dalta_tas_all[gi+N_esm+1][pliter]- dalta_tas_all[gi+1][pliter],
                 markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])

    plt.ylim([-1.5,1.5])

plt.tight_layout()
plt.figure(figsize = (15,4))

for pliter in range(8):
    plt.subplot(2,4,pliter +1)
    plt.plot(1,delta_spi_all[0][pliter], marker = markers[0],markersize = 10,linestyle = 'None',mfc='none',label ='Random Fields')

    for gi in range(N_esm):
        plt.plot( 2 , delta_spi_all[gi+1][pliter],
                 markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])

        plt.plot( 3 , delta_spi_all[gi+N_esm+1][pliter],
                 markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])
        
        
        plt.plot( 4 , delta_spi_all[gi+N_esm+1][pliter]- delta_spi_all[gi+1][pliter],
                 markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])
        
    plt.ylim([-1.5,1.5])
plt.tight_layout()
plt.figure(figsize = (15,4))

for pliter in range(8):
    plt.subplot(2,4,pliter +1)
    plt.plot(1,delta_tas_spi_all[0][pliter], marker = markers[0],markersize = 10,linestyle = 'None',mfc='none',label ='Random Fields')

    for gi in range(N_esm):
        plt.plot( 2 , delta_tas_spi_all[gi+1][pliter],
                 markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])

        plt.plot( 3 , delta_tas_spi_all[gi+N_esm+1][pliter],
                 markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])
        
        plt.plot( 4, delta_tas_spi_all[gi+N_esm+1][pliter]- delta_tas_spi_all[gi+1][pliter],
                 markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])
        
    plt.ylim([-1.5,1.5])

    
    

plt.tight_layout()
        



In [ ]:
gcm_iter=3
Dl=[0,1+gcm_iter,5+gcm_iter,9+gcm_iter]
gcm_name=gcm_names[gcm_iter]
dapa=[delta_tas_d1[Dl[0],:,:],
  delta_tas_d1[Dl[1],:,:],
  delta_tas_d1[Dl[2],:,:],
  delta_tas_d1[Dl[3],:,:]]

y_limit=[-1.5,1.5]
labels=['$\degree$C']
lofff=[0,1,0,1]
cofff=[0,0,0,0]
titlesx=['Observation','Raw '+gcm_name,'BCSD '+gcm_name,'BCSD Random']
titlesy=['$\Delta$T|D1']
ex='both'
fig=plt.figure(figsize=(16,8))

for rr in range(2):
  for cc in range(2):
    iter=rr*2+cc
    ax=plt.subplot(2,2,iter+1)
    #plt.pcolor(ma.masked_array(dapa[rr*4+cc], mask=mask))
    map_plot_cyl_final(ma.masked_array(dapa[iter], mask=mask_shape),lat_obs,lon_obs-360,data_bounds,labels[0],y_limits[0],ax,mp_spacing=10,subplot_iter=iter+1,loff=lofff[iter],coff=cofff[iter],ex=ex)
    plt.title(titlesx[iter])


fig.text(0.08, 0.5, titlesy[0], va='center', ha='center', rotation='vertical', fontsize=16)
fig.suptitle(gcm_name)
plt.savefig('/content/drive/MyDrive/Figures/Figure5/brief_'+gcm_name+'.svg')

In [ ]:
data_bounds=[ 360-125, 50, 360-70, 25]

lab_list=['Obs','GCM0*','GCM1*','GCM2*','GCM3*','BCSD-GCM0','BCSD-GCM1','BCSD-GCM2','BCSD-GCM3','BCSD-RND0','BCSD-RND1','BCSD-RND2','BCSD-RND3']
lweight=500
lsize=20
lcol='k'
sx=1
sy=3
fig=plt.figure(figsize=(25,60))
ylim=[250,330]
for ii in range(sx):
  ind=ii*3
  ax=plt.subplot(sx,sy,ii*3+1)
  map_plot_cyl_change(delta_tas[ii],lat_obs,lon_obs,data_bounds,'',ylim,ax,mp_spacing=10,subplot_iter=ii+1,loff=(ind+1)%3,coff=(ind+1)%3)
  plt.title(lab_list[ii]+': $\Delta$T',fontweight=lweight,fontsize=lsize,color=lcol)

  ind=ii*3+1
  ax=plt.subplot(sx,sy,ii*3+1+1) 
  tt0=tas_period_0[ii,:,:,:]

  map_plot_cyl_change(tt0[0,:,:],lat_obs,lon_obs,data_bounds,'',ylim,ax,mp_spacing=10,subplot_iter=ii+1,loff=(ind+1)%3,coff=(ind+1)%3)
  plt.title(lab_list[ii]+': $\Delta$T|D0',fontweight=lweight,fontsize=lsize,color=lcol)
  ind=ii*3+1
  tt1=tas_period_1[ii,:,:,:]
  ax=plt.subplot(sx,sy,ii*3+2+1)
  map_plot_cyl_change(np.nanmean(tt1,axis=0),lat_obs,lon_obs,data_bounds,'',ylim,ax,mp_spacing=10,subplot_iter=ii+1,loff=(ind+1)%3,coff=(ind+1)%3)
  plt.title(lab_list[ii]+': $\Delta$T|D1',fontweight=lweight,fontsize=lsize,color=lcol)

plt.suptitle("Change in Annual Mean: (1902-1951) to (1955-2005)",fontsize=24,color='k')



In [ ]:
data_bounds=[ 360-125, 50, 360-70, 25]

lab_list=['Obs','GCM0*','GCM1*','GCM2*','GCM3*','BCSD-GCM0','BCSD-GCM1','BCSD-GCM2','BCSD-GCM3','BCSD-RND0','BCSD-RND1','BCSD-RND2','BCSD-RND3']
lweight=500
lsize=20
lcol='k'
sx=13
sy=3
fig=plt.figure(figsize=(25,60))
ylim=[250,330]
for ii in range(sx):
  ind=ii*3
  ax=plt.subplot(sx,sy,ii*3+1)
  map_plot_cyl_change(delta_tas[ii],lat_obs,lon_obs,data_bounds,'',ylim,ax,mp_spacing=10,subplot_iter=ind+1,loff=(ind+1)%3,coff=(ind+1)%3)
  plt.title(lab_list[ii]+': $\Delta$T',fontweight=lweight,fontsize=lsize,color=lcol)

  ind=ii*3+1
  ax=plt.subplot(sx,sy,ii*3+1+1) 
  tt0=spi_period_0[ii,:,:,:]

  map_plot_cyl_change(tt0[0,:,:],lat_obs,lon_obs,data_bounds,'',ylim,ax,mp_spacing=10,subplot_iter=ind+1,loff=(ind+1)%3,coff=(ind+1)%3)
  plt.title(lab_list[ii]+': $\Delta$T|D0',fontweight=lweight,fontsize=lsize,color=lcol)
  ind=ii*3+2
  tt1=spi_period_1[ii,:,:,:]
  ax=plt.subplot(sx,sy,ii*3+2+1)
  map_plot_cyl_change(np.nanmean(tt1,axis=0),lat_obs,lon_obs,data_bounds,'',ylim,ax,mp_spacing=10,subplot_iter=ind+1,loff=(ind+1)%3,coff=(ind+1)%3)
  plt.title(lab_list[ii]+': $\Delta$T|D1',fontweight=lweight,fontsize=lsize,color=lcol)

plt.suptitle("Change in Annual Mean: (1902-1951) to (1955-2005)",fontsize=24,color='k')

